In [1]:
import warnings; warnings.simplefilter('ignore')

import re
import dlib
import timeit
import pickle
import random
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
from sklearn import preprocessing
from matplotlib.pyplot import imshow
from collections import defaultdict, Counter

label_encoder = preprocessing.LabelEncoder()
le = preprocessing.LabelEncoder()
oe = preprocessing.OneHotEncoder()

%matplotlib inline

In [2]:
# Number of examples
N = 5000
# Length of a feature
d = 128
# ICM iterations
NITER = 5

In [3]:
def l2i(a):
        return int(ord(a)-ord('a'))
def i2l(i):
    if i >= 0:
        return chr(i+ord('a'))
    else:
        return '_'
def iors(s):
    try:
        return int(s)
    except ValueError: # if it is a string, return a string
        return s

In [4]:
def read_OCR(filename, n_features):
    F = open(filename)
    dataset = {}
    dataset['ids'] = []#np.zeros(n_examples, dtype=int)
    dataset['labels'] = []#np.zeros(n_examples,dtype=int)
    dataset['labelDic'] = {} # To profile the distribution of labels
    dataset['next_ids'] = []#np.zeros(n_examples,dtype=int)
    dataset['word_ids'] = []#np.zeros(n_examples,dtype=int)
    dataset['positions'] = []#np.zeros(n_examples,dtype=int)
    dataset['folds'] = []#np.zeros(n_examples,dtype=int)
    dataset['features'] = []#np.zeros([n_examples,n_features])
    
    for str_line in F.readlines():
        #line0 = map(iors, filter(None, re.split('\t', str_line.strip())))
        ## ATTENTION: If you are using Python3, use the following line instead
        line0 = list(map(iors, filter(None, re.split('\t', str_line.strip()))))


        dataset['ids'].append(int(line0.pop(0)))
        dataset['labels'].append(l2i(line0.pop(0))) # The label is converted into integer('a'=>0, 'z'=>25)
        if dataset['labels'][-1] in dataset['labelDic']:
            dataset['labelDic'][dataset['labels'][-1]] += 1
        else:
            dataset['labelDic'][dataset['labels'][-1]] = 1
            
        dataset['next_ids'].append(int(line0.pop(0)))
        dataset['word_ids'].append(int(line0.pop(0)))
        dataset['positions'].append(int(line0.pop(0)))
        dataset['folds'].append(int(line0.pop(0)))
        if len(line0) != 128:  # Sanity check of the length
            print (len(line0))
        dataset['features'].append(line0)

    return dataset

In [5]:
def structurize(dataset, N, L):
    d_features = len(dataset['features'][0])
    y = dataset['labels']
    X = dataset['features']
    next_id = dataset['next_ids']

    labels = np.zeros((N, L))
    features = np.zeros((N, L*d_features))
    
    # Extract only one structured example
    def extract(iN, loc):
        labels[iN] = y[loc:loc+L]
        features[iN] = np.array(X[loc:loc+L]).ravel().tolist()
        iN += 1
        return iN
    
    iN = 0
    iN = extract(iN, 0)
    
    for key, value in enumerate(y):
        if next_id[key] == -1:
            iN = extract(iN, key+1)
            
            if iN == N:
                break
    
    c = list(zip(labels, features))
    random.shuffle(c)
    labels, features = zip(*c)
    
    return np.array(labels), np.array(features)      

In [6]:
class ThreeClassProblem:
    C = 1

    def __init__(self, samples, labels, L, K, d):
        self.L = L
        self.K = K
        self.d = d
        self.num_samples = len(samples)
        self.num_dimensions = (L*K*d+1) + (L-1)
        self.samples = samples
        self.labels = labels
        self.loss_for_loop = True
        
    def make_psi(self, x, label):       
        psi = dlib.vector()
        psi.resize(self.num_dimensions)
        psi[0] = x[0] # The bias term

        for i in range(self.L):
            for k in range(self.K):
                if (label[i] == k):
                    # i * self.K * self.d + 1= the index of start of current x[i], + 1 due to bias term
                    block_start = i * self.K * self.d + 1
                    # + k * self.d = the index where x[i] starts in its block (due to K off set)
                    offset = k * self.d
                    start_index = block_start + offset
                    for j in range(self.d):
                        psi[start_index + j] = x[i * self.d + 1 + j]
        
        # The last L-1 elements are the transition variables
        for i in range(self.L-1):
            if label[i] != label[i+1]:
                psi[self.K * self.d * self.L + i] = 1.0
            else:
                psi[self.K * self.d * self.L + i] = 0.0
        
        return psi

    def get_truth_joint_feature_vector(self, idx):
        return self.make_psi(self.samples[idx], self.labels[idx])
    
    def separation_oracle(self, idx, current_solution):
        samp = self.samples[idx]
        psi = [0]*self.num_dimensions
        max1 = -1e10
        max_scoring_label = [0]*self.L # Initialize max_scoring_label for icm search
        for k in range(NITER):
            for iL in range(self.L):   # Iterate over the window length
                for i in range(self.K):# Change different label for the search of a structured label
                    tmp_label = max_scoring_label.copy() # New a list to avoid modifying the max_scoring_label
                    tmp_label[iL] = i # Take turns to modify the structured label from left to right. The guessed structured label.
                    tmp_psi = self.make_psi(samp, tmp_label) # Make a new Psi for the guessed structured label
                    score1 = dlib.dot(current_solution, tmp_psi)
                    
                    loss1 = 0.0
                    if self.loss_for_loop:
                        for j in range(self.L):
                            if self.labels[idx][j] != tmp_label[j]:
                                loss1 += 1.0
                    else:
                        if self.labels[idx] != tmp_label: # Add the conditional "1"
                            loss1 += 1.0

                    if max1 < score1+loss1: # Search for the maximum and update loss, max_scoring_label, and psi
                        max1 = score1 + loss1
                        loss = loss1
                        max_scoring_label[iL] = i
                        psi = tmp_psi

        return loss, psi

In [7]:
def predict_words(dataset, L, problem, weights, K):
    words = defaultdict(lambda: {'features': [], 'labels': [], 'next_ids': []})

    # Extract by per word
    for i in range(len(dataset['features'])):
        word_id = dataset['word_ids'][i]
        words[word_id]['features'].append(dataset['features'][i])
        words[word_id]['labels'].append(dataset['labels'][i])
        words[word_id]['next_ids'].append(dataset['next_ids'][i])

    predicted_labels = []
    true_labels = []
    next_ids = []

    # Only consider the first 1000 words for prediction
    words_to_predict = list(words.values())[:1000]

    for word in words_to_predict:
        word_feats = word['features']
        word_labels = word['labels']
        word_next_ids = word['next_ids']
        word_len = len(word_feats)

        if word_len < L:
            continue

        # For each character position, collect {label: list of scores}
        score_votes = [dict() for _ in range(word_len)]

        # Slide window with stride = 1
        for i in range(0, word_len - L + 1):
            x_seq = word_feats[i:i + L]

            # ICM initialization
            prediction = [0] * L

            for _ in range(NITER):
                for iL in range(L):
                    best_label = 0
                    best_score = -1e10
                    for label in range(K):
                        tmp_label = prediction.copy()
                        tmp_label[iL] = label

                        # Flatten and add bias before passing to make_psi
                        x_flat = [1.0] + np.array(x_seq).ravel().tolist()
                        psi = problem.make_psi(x_flat, tmp_label)
                        
                        score = dlib.dot(weights, psi)
                        if score > best_score:
                            best_score = score
                            best_label = label
                    prediction[iL] = best_label

            # Final score for full sequence
            x_flat = [1.0] + np.array(x_seq).ravel().tolist()
            psi_final = problem.make_psi(x_flat, prediction)
            seq_score = dlib.dot(weights, psi_final)

            for j in range(L):
                abs_pos = i + j
                label = prediction[j]
                prev_best = score_votes[abs_pos].get(label, -1e10)
                score_votes[abs_pos][label] = max(prev_best, seq_score)

        # Final prediction: pick label with highest max score at each position
        word_pred = []
        for label_score_dict in score_votes:
            if not label_score_dict:
                word_pred.append(0)
            else:
                best_label = max(label_score_dict.items(), key=lambda x: x[1])[0]
                word_pred.append(best_label)

        predicted_labels.extend(word_pred)
        true_labels.extend(word_labels)
        next_ids.extend(word_next_ids)

    return predicted_labels, true_labels, next_ids

In [8]:
def full_word_accuracy(predicted_labels, true_labels, next_ids):
    assert len(predicted_labels) == len(true_labels) == len(next_ids)

    pred_words = []
    true_words = []

    pred_buffer = []
    true_buffer = []

    for pred, truth, nxt in zip(predicted_labels, true_labels, next_ids):
        pred_buffer.append(pred)
        true_buffer.append(truth)

        if nxt == -1:
            pred_words.append(pred_buffer)
            true_words.append(true_buffer)
            pred_buffer = []
            true_buffer = []

    assert len(pred_words) == len(true_words)

    correct = sum(p == t for p, t in zip(pred_words, true_words))
    total = len(true_words)
    return correct / total if total > 0 else 0.0

In [9]:
def cal_accuracy(samples, labels, problem, weights, K):
    predictions = []
    for samp in samples:
        prediction = [0]*problem.L # Initialize max_scoring_label for icm search
        for k in range(NITER):
            for iL in range(problem.L):
                max = -1e10
                for i in range(K):
                    tmp_label = prediction.copy()
                    tmp_label[iL] = i
                    psi1 = problem.make_psi(samp, tmp_label)
                    score = dlib.dot(weights, psi1)

                    if max < score:
                        max = score
                        prediction[iL] = i
        predictions.append(prediction)
    
    
    errCnt = 0
    for i in range(len(predictions)):
        if predictions[i] != labels[i]:
            errCnt += 1

    return 1.0-float(errCnt)/float(len(predictions))

In [10]:
dataset = read_OCR('OCRdataset/letter.data', d)

## L = 1

In [11]:
L = 1

### 4000/1000

In [12]:
# Setup
labels, features = structurize(dataset, N, L)
TRAIN = 0.8
model_path = 'weights1_82.obj'

In [13]:
le = preprocessing.LabelEncoder()
nplabels  = le.fit_transform(labels.ravel()).reshape(labels.shape)
npsamples = np.hstack([np.ones((N,1)), features]) # Add ones for bias
K = len(le.classes_)
print ('K=', K)

tr_labels  = nplabels[:int(N*TRAIN)].astype(int).tolist()
tr_samples = npsamples[:int(N*TRAIN)].astype(int).tolist()
te_labels  = nplabels[int(N*TRAIN):].astype(int).tolist()
te_samples = npsamples[int(N*TRAIN):].astype(int).tolist()

problem = ThreeClassProblem(tr_samples, tr_labels, L, K, d)

K= 13


In [14]:
start_train = timeit.default_timer()
weights = dlib.solve_structural_svm_problem(problem)
end_train = timeit.default_timer()
print ("Training time elapsed:", end_train - start_train, "s")
pickle.dump(weights, open(model_path, 'wb'))

Training time elapsed: 91.16748304199973 s


In [15]:
weights_load = pickle.load(open(model_path,'rb'))

print ("Training accuracy=", cal_accuracy(tr_samples, tr_labels, problem, weights_load, K))
start_test = timeit.default_timer()
print ("Test accuracy=", cal_accuracy(te_samples, te_labels, problem, weights_load, K))
end_test = timeit.default_timer()
print ("Testing time elapsed:", end_test - start_test, "s")

start_predict = timeit.default_timer()
predicted_labels, true_labels, next_ids = predict_words(dataset, L, problem, weights_load, K)
end_predict = timeit.default_timer()
print ("Prediction time elapsed:", end_predict - start_predict, "s")
full_accuracy = full_word_accuracy(predicted_labels, true_labels, next_ids)
print ("Full word accuracy:", full_accuracy)

Training accuracy= 0.6945
Test accuracy= 0.6859999999999999
Testing time elapsed: 2.735049874998367 s
Prediction time elapsed: 23.564398582999274 s
Full word accuracy: 0.0


### 2500/2500

In [16]:
# Setup
labels, features = structurize(dataset, N, L)
TRAIN = 0.5
model_path = 'weights1_55.obj'

In [17]:
le = preprocessing.LabelEncoder()
nplabels  = le.fit_transform(labels.ravel()).reshape(labels.shape)
npsamples = np.hstack([np.ones((N,1)), features]) # Add ones for bias
K = len(le.classes_)
print ('K=', K)

tr_labels  = nplabels[:int(N*TRAIN)].astype(int).tolist()
tr_samples = npsamples[:int(N*TRAIN)].astype(int).tolist()
te_labels  = nplabels[int(N*TRAIN):].astype(int).tolist()
te_samples = npsamples[int(N*TRAIN):].astype(int).tolist()

problem = ThreeClassProblem(tr_samples, tr_labels, L, K, d)

K= 13


In [18]:
start_train = timeit.default_timer()
weights = dlib.solve_structural_svm_problem(problem)
end_train = timeit.default_timer()
print ("Training time elapsed:", end_train - start_train, "s")
pickle.dump(weights, open(model_path, 'wb'))

Training time elapsed: 55.737573874999725 s


In [19]:
weights_load = pickle.load(open(model_path,'rb'))

print ("Training accuracy=", cal_accuracy(tr_samples, tr_labels, problem, weights_load, K))
start_test = timeit.default_timer()
print ("Test accuracy=", cal_accuracy(te_samples, te_labels, problem, weights_load, K))
end_test = timeit.default_timer()
print ("Testing time elapsed:", end_test - start_test, "s")

start_predict = timeit.default_timer()
predicted_labels, true_labels, next_ids = predict_words(dataset, L, problem, weights_load, K)
end_predict = timeit.default_timer()
print ("Prediction time elapsed:", end_predict - start_predict, "s")
full_accuracy = full_word_accuracy(predicted_labels, true_labels, next_ids)
print ("Full word accuracy:", full_accuracy)

Training accuracy= 0.7048
Test accuracy= 0.6839999999999999
Testing time elapsed: 6.795320833000005 s
Prediction time elapsed: 23.621400500000163 s
Full word accuracy: 0.0


#### 1000/4000

In [20]:
# Setup
labels, features = structurize(dataset, N, L)
TRAIN = 0.2
model_path = 'weights1_28.obj'

In [21]:
le = preprocessing.LabelEncoder()
nplabels  = le.fit_transform(labels.ravel()).reshape(labels.shape)
npsamples = np.hstack([np.ones((N,1)), features]) # Add ones for bias
K = len(le.classes_)
print ('K=', K)

tr_labels  = nplabels[:int(N*TRAIN)].astype(int).tolist()
tr_samples = npsamples[:int(N*TRAIN)].astype(int).tolist()
te_labels  = nplabels[int(N*TRAIN):].astype(int).tolist()
te_samples = npsamples[int(N*TRAIN):].astype(int).tolist()

problem = ThreeClassProblem(tr_samples, tr_labels, L, K, d)

K= 13


In [22]:
start_train = timeit.default_timer()
weights = dlib.solve_structural_svm_problem(problem)
end_train = timeit.default_timer()
print ("Training time elapsed:", end_train - start_train, "s")
pickle.dump(weights, open(model_path, 'wb'))

Training time elapsed: 22.375254915999903 s


In [23]:
weights_load = pickle.load(open(model_path,'rb'))

print ("Training accuracy=", cal_accuracy(tr_samples, tr_labels, problem, weights_load, K))
start_test = timeit.default_timer()
print ("Test accuracy=", cal_accuracy(te_samples, te_labels, problem, weights_load, K))
end_test = timeit.default_timer()
print ("Testing time elapsed:", end_test - start_test, "s")

start_predict = timeit.default_timer()
predicted_labels, true_labels, next_ids = predict_words(dataset, L, problem, weights_load, K)
end_predict = timeit.default_timer()
print ("Prediction time elapsed:", end_predict - start_predict, "s")
full_accuracy = full_word_accuracy(predicted_labels, true_labels, next_ids)
print ("Full word accuracy:", full_accuracy)

Training accuracy= 0.726
Test accuracy= 0.67025
Testing time elapsed: 10.902206750000914 s
Prediction time elapsed: 23.61164245800137 s
Full word accuracy: 0.0


## L = 2

In [24]:
L = 2

### 4000/1000

In [ ]:
# Setup
labels, features = structurize(dataset, N, L)
TRAIN = 0.8
model_path = 'weights2_82.obj'

In [26]:
le = preprocessing.LabelEncoder()
nplabels  = le.fit_transform(labels.ravel()).reshape(labels.shape)
npsamples = np.hstack([np.ones((N,1)), features]) # Add ones for bias
K = len(le.classes_)
print ('K=', K)

tr_labels  = nplabels[:int(N*TRAIN)].astype(int).tolist()
tr_samples = npsamples[:int(N*TRAIN)].astype(int).tolist()
te_labels  = nplabels[int(N*TRAIN):].astype(int).tolist()
te_samples = npsamples[int(N*TRAIN):].astype(int).tolist()

problem = ThreeClassProblem(tr_samples, tr_labels, L, K, d)

K= 23


In [27]:
start_train = timeit.default_timer()
weights = dlib.solve_structural_svm_problem(problem)
end_train = timeit.default_timer()
print ("Training time elapsed:", end_train - start_train, "s")
pickle.dump(weights, open(model_path, 'wb'))

Training time elapsed: 1034.7579585420008 s


In [28]:
weights_load = pickle.load(open(model_path,'rb'))

print ("Training accuracy=", cal_accuracy(tr_samples, tr_labels, problem, weights_load, K))
start_test = timeit.default_timer()
print ("Test accuracy=", cal_accuracy(te_samples, te_labels, problem, weights_load, K))
end_test = timeit.default_timer()
print ("Testing time elapsed:", end_test - start_test, "s")

start_predict = timeit.default_timer()
predicted_labels, true_labels, next_ids = predict_words(dataset, L, problem, weights_load, K)
end_predict = timeit.default_timer()
print ("Prediction time elapsed:", end_predict - start_predict, "s")
full_accuracy = full_word_accuracy(predicted_labels, true_labels, next_ids)
print ("Full word accuracy:", full_accuracy)

Training accuracy= 0.481
Test accuracy= 0.43999999999999995
Testing time elapsed: 19.073470875000567 s
Prediction time elapsed: 141.46359883399964 s
Full word accuracy: 0.0


### 2500/2500

In [ ]:
# Setup
labels, features = structurize(dataset, N, L)
TRAIN = 0.5
model_path = 'weights2_55.obj'

In [30]:
le = preprocessing.LabelEncoder()
nplabels  = le.fit_transform(labels.ravel()).reshape(labels.shape)
npsamples = np.hstack([np.ones((N,1)), features]) # Add ones for bias
K = len(le.classes_)
print ('K=', K)

tr_labels  = nplabels[:int(N*TRAIN)].astype(int).tolist()
tr_samples = npsamples[:int(N*TRAIN)].astype(int).tolist()
te_labels  = nplabels[int(N*TRAIN):].astype(int).tolist()
te_samples = npsamples[int(N*TRAIN):].astype(int).tolist()

problem = ThreeClassProblem(tr_samples, tr_labels, L, K, d)

K= 23


In [31]:
start_train = timeit.default_timer()
weights = dlib.solve_structural_svm_problem(problem)
end_train = timeit.default_timer()
print ("Training time elapsed:", end_train - start_train, "s")
pickle.dump(weights, open(model_path, 'wb'))

Training time elapsed: 633.6864789580013 s


In [32]:
weights_load = pickle.load(open(model_path,'rb'))

print ("Training accuracy=", cal_accuracy(tr_samples, tr_labels, problem, weights_load, K))
start_test = timeit.default_timer()
print ("Test accuracy=", cal_accuracy(te_samples, te_labels, problem, weights_load, K))
end_test = timeit.default_timer()
print ("Testing time elapsed:", end_test - start_test, "s")

start_predict = timeit.default_timer()
predicted_labels, true_labels, next_ids = predict_words(dataset, L, problem, weights_load, K)
end_predict = timeit.default_timer()
print ("Prediction time elapsed:", end_predict - start_predict, "s")
full_accuracy = full_word_accuracy(predicted_labels, true_labels, next_ids)
print ("Full word accuracy:", full_accuracy)

Training accuracy= 0.5032
Test accuracy= 0.4468
Testing time elapsed: 47.776799582999956 s
Prediction time elapsed: 141.3395536249991 s
Full word accuracy: 0.0


#### 1000/4000

In [ ]:
# Setup
labels, features = structurize(dataset, N, L)
TRAIN = 0.2
model_path = 'weights2_28.obj'

In [34]:
le = preprocessing.LabelEncoder()
nplabels  = le.fit_transform(labels.ravel()).reshape(labels.shape)
npsamples = np.hstack([np.ones((N,1)), features]) # Add ones for bias
K = len(le.classes_)
print ('K=', K)

tr_labels  = nplabels[:int(N*TRAIN)].astype(int).tolist()
tr_samples = npsamples[:int(N*TRAIN)].astype(int).tolist()
te_labels  = nplabels[int(N*TRAIN):].astype(int).tolist()
te_samples = npsamples[int(N*TRAIN):].astype(int).tolist()

problem = ThreeClassProblem(tr_samples, tr_labels, L, K, d)

K= 23


In [35]:
start_train = timeit.default_timer()
weights = dlib.solve_structural_svm_problem(problem)
end_train = timeit.default_timer()
print ("Training time elapsed:", end_train - start_train, "s")
pickle.dump(weights, open(model_path, 'wb'))

Training time elapsed: 224.20808962499905 s


In [36]:
weights_load = pickle.load(open(model_path,'rb'))

print ("Training accuracy=", cal_accuracy(tr_samples, tr_labels, problem, weights_load, K))
start_test = timeit.default_timer()
print ("Test accuracy=", cal_accuracy(te_samples, te_labels, problem, weights_load, K))
end_test = timeit.default_timer()
print ("Testing time elapsed:", end_test - start_test, "s")

start_predict = timeit.default_timer()
predicted_labels, true_labels, next_ids = predict_words(dataset, L, problem, weights_load, K)
end_predict = timeit.default_timer()
print ("Prediction time elapsed:", end_predict - start_predict, "s")
full_accuracy = full_word_accuracy(predicted_labels, true_labels, next_ids)
print ("Full word accuracy:", full_accuracy)

Training accuracy= 0.5840000000000001
Test accuracy= 0.44725000000000004
Testing time elapsed: 76.43969287499931 s
Prediction time elapsed: 141.92162695800107 s
Full word accuracy: 0.0


## L = 3

In [37]:
L = 3

### 4000/1000

In [ ]:
# Setup
labels, features = structurize(dataset, N, L)
TRAIN = 0.8
model_path = 'weights3_82.obj'

In [39]:
le = preprocessing.LabelEncoder()
nplabels  = le.fit_transform(labels.ravel()).reshape(labels.shape)
npsamples = np.hstack([np.ones((N,1)), features]) # Add ones for bias
K = len(le.classes_)
print ('K=', K)

tr_labels  = nplabels[:int(N*TRAIN)].astype(int).tolist()
tr_samples = npsamples[:int(N*TRAIN)].astype(int).tolist()
te_labels  = nplabels[int(N*TRAIN):].astype(int).tolist()
te_samples = npsamples[int(N*TRAIN):].astype(int).tolist()

problem = ThreeClassProblem(tr_samples, tr_labels, L, K, d)

K= 24


In [40]:
start_train = timeit.default_timer()
weights = dlib.solve_structural_svm_problem(problem)
end_train = timeit.default_timer()
print ("Training time elapsed:", end_train - start_train, "s")
pickle.dump(weights, open(model_path, 'wb'))

Training time elapsed: 2242.6402945829996 s


In [41]:
weights_load = pickle.load(open(model_path,'rb'))

print ("Training accuracy=", cal_accuracy(tr_samples, tr_labels, problem, weights_load, K))
start_test = timeit.default_timer()
print ("Test accuracy=", cal_accuracy(te_samples, te_labels, problem, weights_load, K))
end_test = timeit.default_timer()
print ("Testing time elapsed:", end_test - start_test, "s")

start_predict = timeit.default_timer()
predicted_labels, true_labels, next_ids = predict_words(dataset, L, problem, weights_load, K)
end_predict = timeit.default_timer()
print ("Prediction time elapsed:", end_predict - start_predict, "s")
full_accuracy = full_word_accuracy(predicted_labels, true_labels, next_ids)
print ("Full word accuracy:", full_accuracy)

Training accuracy= 0.3335
Test accuracy= 0.30500000000000005
Testing time elapsed: 45.23605624999982 s
Prediction time elapsed: 279.48097662500004 s
Full word accuracy: 0.0


### 2500/2500

In [ ]:
# Setup
labels, features = structurize(dataset, N, L)
TRAIN = 0.5
model_path = 'weights3_55.obj'

In [43]:
le = preprocessing.LabelEncoder()
nplabels  = le.fit_transform(labels.ravel()).reshape(labels.shape)
npsamples = np.hstack([np.ones((N,1)), features]) # Add ones for bias
K = len(le.classes_)
print ('K=', K)

tr_labels  = nplabels[:int(N*TRAIN)].astype(int).tolist()
tr_samples = npsamples[:int(N*TRAIN)].astype(int).tolist()
te_labels  = nplabels[int(N*TRAIN):].astype(int).tolist()
te_samples = npsamples[int(N*TRAIN):].astype(int).tolist()

problem = ThreeClassProblem(tr_samples, tr_labels, L, K, d)

K= 24


In [44]:
start_train = timeit.default_timer()
weights = dlib.solve_structural_svm_problem(problem)
end_train = timeit.default_timer()
print ("Training time elapsed:", end_train - start_train, "s")
pickle.dump(weights, open(model_path, 'wb'))

Training time elapsed: 1599.3623139170013 s


In [45]:
weights_load = pickle.load(open(model_path,'rb'))

print ("Training accuracy=", cal_accuracy(tr_samples, tr_labels, problem, weights_load, K))
start_test = timeit.default_timer()
print ("Test accuracy=", cal_accuracy(te_samples, te_labels, problem, weights_load, K))
end_test = timeit.default_timer()
print ("Testing time elapsed:", end_test - start_test, "s")

start_predict = timeit.default_timer()
predicted_labels, true_labels, next_ids = predict_words(dataset, L, problem, weights_load, K)
end_predict = timeit.default_timer()
print ("Prediction time elapsed:", end_predict - start_predict, "s")
full_accuracy = full_word_accuracy(predicted_labels, true_labels, next_ids)
print ("Full word accuracy:", full_accuracy)

Training accuracy= 0.34840000000000004
Test accuracy= 0.2964
Testing time elapsed: 114.93170691699925 s
Prediction time elapsed: 287.01649533300224 s
Full word accuracy: 0.0


#### 1000/4000

In [46]:
# Setup
labels, features = structurize(dataset, N, L)
TRAIN = 0.2
model_path = 'weights3_28.obj'

In [47]:
le = preprocessing.LabelEncoder()
nplabels  = le.fit_transform(labels.ravel()).reshape(labels.shape)
npsamples = np.hstack([np.ones((N,1)), features]) # Add ones for bias
K = len(le.classes_)
print ('K=', K)

tr_labels  = nplabels[:int(N*TRAIN)].astype(int).tolist()
tr_samples = npsamples[:int(N*TRAIN)].astype(int).tolist()
te_labels  = nplabels[int(N*TRAIN):].astype(int).tolist()
te_samples = npsamples[int(N*TRAIN):].astype(int).tolist()

problem = ThreeClassProblem(tr_samples, tr_labels, L, K, d)

K= 24


In [48]:
start_train = timeit.default_timer()
weights = dlib.solve_structural_svm_problem(problem)
end_train = timeit.default_timer()
print ("Training time elapsed:", end_train - start_train, "s")
pickle.dump(weights, open(model_path, 'wb'))

Training time elapsed: 672.6627867909992 s


In [49]:
weights_load = pickle.load(open(model_path,'rb'))

print ("Training accuracy=", cal_accuracy(tr_samples, tr_labels, problem, weights_load, K))
start_test = timeit.default_timer()
print ("Test accuracy=", cal_accuracy(te_samples, te_labels, problem, weights_load, K))
end_test = timeit.default_timer()
print ("Testing time elapsed:", end_test - start_test, "s")

start_predict = timeit.default_timer()
predicted_labels, true_labels, next_ids = predict_words(dataset, L, problem, weights_load, K)
end_predict = timeit.default_timer()
print ("Prediction time elapsed:", end_predict - start_predict, "s")
full_accuracy = full_word_accuracy(predicted_labels, true_labels, next_ids)
print ("Full word accuracy:", full_accuracy)

Training accuracy= 0.398
Test accuracy= 0.25775000000000003
Testing time elapsed: 178.97353929200108 s
Prediction time elapsed: 283.40860166700077 s
Full word accuracy: 0.0
